In [1]:
!pip install lifelines
!pip install pyckmeans
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 11.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=078cb368ed305eead9990cc2e600aeb8c42674d0f80bb0e2c0b60631481ceb51
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 890.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-cp310-cp310-linux_x86_64.whl size=93213 sha256=f362d922dd348f00398cffd7180dcb5e7b57164afcbf24d617ad9e097d14cd6f
  Stored in directory: /root/.cache/pip/wheels/c3/83/bb/d0fc31d29f361aff1f400ca2fb2d32c98f900e1aa480229f99
Successfully built pyckmeans
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 M

In [2]:
import numpy as np
import pandas as pd
from pyckmeans import CKmeans
import matplotlib.pyplot as plt

from lifelines.fitters.coxph_fitter import CoxPHFitter

from google.colab import drive

import wandb
import os

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load raw BRCA data ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
def drop_elements_from_cluster(cluster, consensus_matrix):
  new_cluster = []
  for j in range(0, len(cluster)):
    if consensus_matrix[cluster[0]][cluster[j]] == 1:
      new_cluster.append(cluster[j])
  return new_cluster

In [6]:
def convert_indices_to_binary_list(number_elements, cluster_0, cluster_1):
  cluster0_column = []
  cluster1_column = []
  for k in range(0, number_elements):
    if k in cluster_0:
      cluster0_column.append(1)
      cluster1_column.append(0)
    elif k in cluster_1:
      cluster0_column.append(0)
      cluster1_column.append(1)
    else:
      cluster0_column.append(0)
      cluster1_column.append(0)

  return [cluster0_column, cluster1_column]

In [7]:
def fit_cox_model(cluster, data_frame):
  print("Original dataframe shape: ", data_frame.shape)
  cluster_frame = pd.DataFrame({'cluster': cluster})
  # Concatenate cluster membership to data table
  if 'cluster' in data_frame.columns.values:
    data_frame = data_frame.drop('cluster', axis = 1) # Drop columns that could have been created previously
  data_frame = pd.concat([data_frame, cluster_frame], axis = 1)

  # Get rows with null entry
  null_mask = data_frame.isnull().any(axis=1)
  null_rows = data_frame[null_mask]
  print(null_rows)

  # Remove rows with null entry
  data_frame.dropna(inplace=True) # remove all rows with any null value
  print("Final dataframe shape: ", data_frame.shape)

  # Fit Cox model
  cph = CoxPHFitter()
  cph.fit(data_frame, duration_col = 'OS.time', event_col = 'OS')
  cph.print_summary()

  # exp(coef) <=> hazard ratio // AN ESTIMATE OF THE TRUE HAZARD RATIO. IT HAS A STANDARD ERROR ASSOCIATED WITH IT.
  # A one unit increase in the covariate will increase the hazard by the hazard ratio

  # Every coefficient comes with a p-value
  # The p-value represents the probability of observing this coefficient in a sample if the null hypothesis was true.
  # The null hypothesis states that the coefficient=0, meaning that the predictor variable does not influence the hazard rate (occurence of the event)

  # The CoxPHFitter computes p-values using the chi-squared test.
  # The reference is in "Survival Analysis by John P. Klein and Melvin L. Moeschberger, Second Edition", page 256

  p_values = cph._compute_p_values()
  hazard_ratios = cph.hazard_ratios_.tolist()
  coefficients_ci = cph.confidence_intervals_
  print(p_values)
  print(hazard_ratios)
  print(coefficients_ci)
  print("p-value:", p_values[4])
  print("hazard-ratio:", hazard_ratios[4])

  return p_values[4], hazard_ratios[4]

In [ ]:
for i in range (4, 5):
  data_frame = pd.read_csv("TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv", sep="\t")
  data_frame = data_frame[['age', 'OS', 'OS.time', 'stage_2', 'stage_3', 'stage_4']]

  #name_embedding = "UMAP_EM"
  name_embedding = "VAE_Z_EM_Mean_{i}".format(i=i)
  embeddings = np.loadtxt(name_embedding+".txt")
  print("Embeddings shape: ", embeddings.shape)
  print("------------- i =", i, "--------------------")
  number_of_clusters = 2
  ckm = CKmeans(k=number_of_clusters, n_rep=20, p_samp=1, p_feat=1)
  ckm.fit(embeddings)
  ckm_results = ckm.predict(embeddings, linkage_type='average')
  print(ckm_results.cl)
  consensus_matrix = ckm_results.cmatrix
  print(consensus_matrix)

  cluster_0 = []
  cluster_1 = []
  for j in range(0, ckm_results.cl.shape[0]):
    if ckm_results.cl[j] == 1:
      cluster_1.append(j)
    elif ckm_results.cl[j] == 0:
      cluster_0.append(j)
  #print(cluster_0)
  #print(cluster_1)
  print("Cluster sizes: ", len(cluster_0), ", ", len(cluster_1))
  print("Sum of cluster sizes: ", len(cluster_0)+len(cluster_1))

  cluster_0 = drop_elements_from_cluster(cluster_0, consensus_matrix)
  cluster_1 = drop_elements_from_cluster(cluster_1, consensus_matrix)
  print("Cluster sizes after dropping:", len(cluster_0), ", ", len(cluster_1))

  cluster_column_list = convert_indices_to_binary_list(embeddings.shape[0], cluster_0, cluster_1)
  print("Column_lengths: ",  len(cluster_column_list[0]))

  for j in range(0, number_of_clusters):
    if cluster_column_list[j].count(1) > 10:
      print("Cluster size in column form:", cluster_column_list[j].count(1))
      p_value, hazard_ratio = fit_cox_model(cluster_column_list[j], data_frame)
      print("p-value:", p_value)
      print("hazard ratio:", hazard_ratio)
      cluster_name = "{base}_{j}".format(base=name_embedding, j=j)
      run = wandb.init(project="SURVIVAL_ANALAYSIS_k=2", # Set the project where this run will be logged
                       name=cluster_name
      )

      wandb.log({
                "p_value": p_value,
                "hazard_ratio": hazard_ratio,
                "cluster_size": cluster_column_list[j].count(1)
            })

      if p_value < 0.0012:
        file_name = cluster_name + ".txt"
        np.savetxt(file_name, cluster_column_list[j])

Embeddings shape:  (1079, 10)
------------- i = 4 --------------------
[1 1 1 ... 1 1 1]
[[1.   0.5  0.65 ... 0.6  0.7  0.6 ]
 [0.5  1.   0.85 ... 0.8  0.8  0.8 ]
 [0.65 0.85 1.   ... 0.95 0.95 0.95]
 ...
 [0.6  0.8  0.95 ... 1.   0.9  1.  ]
 [0.7  0.8  0.95 ... 0.9  1.   0.9 ]
 [0.6  0.8  0.95 ... 1.   0.9  1.  ]]
Cluster sizes:  259 ,  820
Sum of cluster sizes:  1079
Cluster sizes after dropping: 15 ,  1
Column_lengths:  1079
Cluster size in column form: 15
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0
397   43.0 

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -748.77
         time fit was run = 2024-03-08 19:25:23 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.64       1.89       0.28             0.08             1.20                 1.08                 3.31
stage_3     1.36       3.89       0.30             0.77             1.95                 2.16                 7.01
stage_4     2.47      11.83       0.37             1.74             3.20                 5.71                24.52
cluster     0.51       1.66       0.52            -0.51             1.52                 0.60                 4.58

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.28 <0.005      22.90
stage_2       0.00 2.25   0.02       5.34
stage_3       0.00 4.52 <0.005      17.31
stage_4       0.00 6.65 <0.005      34.99
cluster       0.00 0.97   0.33       1.60
---
Concordance = 0.76
Partial AIC = 1507.54
log-likelihood ratio test = 76.39 on 5 df
-log2(p) of ll-ratio test = 47.57

[1.27937660e-07 2.46725522e-02 6.13407344e-06 2.93163298e-11
 3.29701350e-01]
[1.0363999772233508, 1.8942743201874617, 3.8899519597219716, 11.834894840355933, 1.6576875396169766]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.022486         0.049020
stage_2           0.081480         1.196192
stage_3           0.769592         1.947202
stage_4           1.742752         3.199353
cluster          -0.510885         1.521732
p-value: 0.3297013500532214
hazard-ratio: 1.6576875396169766
p-value: 0.3297013500532214
hazard ratio: 1.6576875396169766
